In [26]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import yfinance as yf 
from datetime import datetime
import matplotlib.pyplot as plt
import scipy.special as sc

In [19]:
from data_gene import GBMsimulator
from monte_carlo import mc_simu
#from price_extractor import select_stock, price_extract, mean_cov  
from calculator import calculate_assets_expectedreturns, calculate_assets_covariance

## implementation steps(scenario 1)
# 1. conduct subpool (option)
# 2. estimate the square of maximum sharpe ratio by theta_hat in 2.32, and compute the response r_c_hat in 2.10
# 3. select lambda through CV according to 2.5.2, demoted by lambda_hat
# 4. set lambda in 2.12 to be lambda_hat and solve for the MAAXSER portfolio omega_hat_star in 2.12
# compare with benchmark portfolio 

In [3]:
# Parameters settings
sigma = 0.04 # variance constraint
T = 120 # sample size
replica = 1000 # replication times
# theoratical maximum SR = 1.882
N = 50 # num of stocks
n_stocks_available = 500

In [17]:
data1 = pd.read_csv("./archive/individual_stocks_5yr/individual_stocks_5yr/AAL_data.csv")
data2 = pd.read_csv("./archive/individual_stocks_5yr/individual_stocks_5yr/AMD_data.csv")
data3 = pd.read_csv("./archive/individual_stocks_5yr/individual_stocks_5yr/EBAY_data.csv")
data4 = pd.read_csv("./archive/individual_stocks_5yr/individual_stocks_5yr/ES_data.csv")
data5 = pd.read_csv("./archive/individual_stocks_5yr/individual_stocks_5yr/IRM_data.csv")
data = pd.DataFrame({"1":data1["close"],"2": data2["close"],"3": data3["close"],"4": data4["close"],"5": data5["close"]})

1    38.393252
2     5.601863
3    42.332582
4    51.285713
5    33.550075
dtype: float64

In [22]:
meanReturns = calculate_assets_expectedreturns(data)        
covMatrix = calculate_assets_covariance(data)

,1,2,3,4,5
1,30257.585103,4665.363018,-10185.140621,13180.566502,4664.443308
2,4665.363018,3975.307083,-4245.984152,4889.665836,1777.997202
3,-10185.140621,-4245.984152,39924.686868,-14407.351044,-1015.700312
4,13180.566502,4889.665836,-14407.351044,12214.710752,4422.782404
5,4664.443308,1777.997202,-1015.700312,4422.782404,4814.138613


In [31]:
temp = np.dot(np.transpose(np.array(meanReturns)), np.linalg.inv(np.array(covMatrix)))
theta_s = np.dot(temp, np.array(meanReturns))
theta_adj_hat = ((T-N-2)*theta_s-N)/T + (2*theta_s**(N/2)*(1+theta_s)**(-(T-2)/2))/(T*sc.betainc(N/2,(T-N)/2, theta_s/(1+theta_s)))

74205.34372291196

In [32]:
r_c = sigma*(1+theta_adj_hat)/np.sqrt(theta_adj_hat)

print(r_c)

8.202563319687986
